In [1]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
import os
import time
import datetime
from scnn_model import SCNN_MODEL
from tensorflow.contrib import learn
from six.moves import cPickle as pickle

In [2]:
# Load vocabulary and the word2vec model
pickle_file = 'save.pickle'
with open(pickle_file, 'rb') as f :
    save = pickle.load(f)
    wordsVectors = save['wordsVectors']
    vocabulary = save['vocabulary']
    del save  # hint to help gc free up memory
print('Vocabulary and the word2vec loaded')
print('Vocabulary size is ', len(vocabulary))
print('Word2Vec model shape is ', wordsVectors.shape)

#Load training data, training labels, validation data, validation labels
pickle_file = 'data_saved.pickle'
with open(pickle_file, 'rb') as f :
    save = pickle.load(f)
    train_data = save['train_data']
    train_labels = save['train_labels']
    validation_data = save['validation_data']
    validation_labels = save['validation_labels']
    del save  # hint to help gc free up memory
print('train data shape ', train_data.shape)
print('train labels shape ', train_labels.shape)
print('validation data shape ', validation_data.shape)
print('validation labels shape ', validation_labels.shape)

Vocabulary and the word2vec loaded
Vocabulary size is  9687
Word2Vec model shape is  (9687, 300)
train data shape  (1280, 160)
train labels shape  (1280, 2)
validation data shape  (160, 160)
validation labels shape  (160, 2)


In [3]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    Original taken from https://github.com/dennybritz/cnn-text-classification-tf/blob/master/data_helpers.py
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [4]:
# Model Hyperparameters
SENTENCE_PER_REVIEW = 16
WORDS_PER_SENTENCE = 10
EMBEDDING_DIM = 300
FILTER_WIDTHS_SENT_CONV = np.array([3, 4, 5])
NUM_FILTERS_SENT_CONV = 100
FILTER_WIDTHS_DOC_CONV = np.array([3, 4, 5])
NUM_FILTERS_DOC_CONV = 100
NUM_CLASSES = 2
DROPOUT_KEEP_PROB = 0.5
L2_REG_LAMBDA = 0.0
BATCH_SIZE = 64
NUM_EPOCHS = 100
EVALUATE_EVERY = 100   # Evaluate model on the validation set after 100 steps
CHECKPOINT_EVERY = 100 # Save model after each 200 steps
NUM_CHECKPOINTS = 5    # Keep only the 5 most recents checkpoints
LEARNING_RATE = 1e-3   # The learning rate

In [5]:
'''Training the model'''
with tf.Graph().as_default():
    sess = tf.Session()
    with sess.as_default():
        cnn = SCNN_MODEL(sentence_per_review=SENTENCE_PER_REVIEW, 
                        words_per_sentence=WORDS_PER_SENTENCE, 
                        wordVectors=wordsVectors, 
                        embedding_size=EMBEDDING_DIM, 
                        filter_widths_sent_conv=FILTER_WIDTHS_SENT_CONV, 
                        num_filters_sent_conv=NUM_FILTERS_SENT_CONV, 
                        filter_widths_doc_conv=FILTER_WIDTHS_DOC_CONV, 
                        num_filters_doc_conv=NUM_FILTERS_DOC_CONV, 
                        num_classes=NUM_CLASSES, 
                        l2_reg_lambda=L2_REG_LAMBDA)
        
        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        
        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))
        
        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)
        
        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)
        
        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)
        
        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=NUM_CHECKPOINTS)
        
        # Initialize all variables
        sess.run(tf.global_variables_initializer())
        
        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
                cnn.input_x: x_batch,
                cnn.input_y: y_batch,
                cnn.input_size: len(y_batch),
                cnn.dropout: DROPOUT_KEEP_PROB
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)
        
        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
                cnn.input_x: x_batch,
                cnn.input_y: y_batch,
                cnn.input_size: y_batch.shape[0],
                cnn.dropout: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)
        
        print('train data shape ', train_data.shape)
        print('train labels shape ', train_labels.shape)
        print('validation data shape ', validation_data.shape)
        print('validation labels shape ', validation_labels.shape)
        
        # Generate batches
        batches = batch_iter(
            list(zip(train_data, train_labels)), BATCH_SIZE, NUM_EPOCHS)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % EVALUATE_EVERY == 0:
                print("\nEvaluation:")
                dev_step(validation_data, validation_labels, writer=dev_summary_writer)
                print("")
            if current_step % CHECKPOINT_EVERY == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

Writing to /Users/cosgalaxy/Desktop/cnn/runs/1541903085

train data shape  (1280, 160)
train labels shape  (1280, 2)
validation data shape  (160, 160)
validation labels shape  (160, 2)
2018-11-10T20:24:49.412299: step 1, loss 1.01786, acc 0.5625
2018-11-10T20:24:50.395240: step 2, loss 1.0168, acc 0.46875
2018-11-10T20:24:51.475834: step 3, loss 1.07817, acc 0.546875
2018-11-10T20:24:52.522676: step 4, loss 0.915397, acc 0.53125
2018-11-10T20:24:53.526401: step 5, loss 0.801735, acc 0.59375
2018-11-10T20:24:54.597978: step 6, loss 1.12066, acc 0.546875
2018-11-10T20:24:55.614021: step 7, loss 1.11103, acc 0.359375
2018-11-10T20:24:56.680886: step 8, loss 1.00322, acc 0.546875
2018-11-10T20:24:57.623846: step 9, loss 0.916618, acc 0.515625
2018-11-10T20:24:58.628959: step 10, loss 1.13323, acc 0.4375
2018-11-10T20:24:59.731031: step 11, loss 0.825115, acc 0.578125
2018-11-10T20:25:00.767253: step 12, loss 1.06468, acc 0.484375
2018-11-10T20:25:01.809932: step 13, loss 0.973223, acc 0.57

2018-11-10T20:27:13.439011: step 124, loss 0.911184, acc 0.515625
2018-11-10T20:27:14.608795: step 125, loss 0.797816, acc 0.625
2018-11-10T20:27:15.764651: step 126, loss 0.809573, acc 0.5625
2018-11-10T20:27:16.905261: step 127, loss 0.778752, acc 0.578125
2018-11-10T20:27:18.170757: step 128, loss 0.890605, acc 0.578125
2018-11-10T20:27:19.385612: step 129, loss 0.75235, acc 0.625
2018-11-10T20:27:20.525475: step 130, loss 0.677846, acc 0.65625
2018-11-10T20:27:21.673538: step 131, loss 0.813809, acc 0.59375
2018-11-10T20:27:22.843750: step 132, loss 0.908745, acc 0.515625
2018-11-10T20:27:23.958671: step 133, loss 0.781506, acc 0.53125
2018-11-10T20:27:25.033193: step 134, loss 0.898011, acc 0.484375
2018-11-10T20:27:26.093645: step 135, loss 0.76958, acc 0.609375
2018-11-10T20:27:27.137190: step 136, loss 0.898569, acc 0.5625
2018-11-10T20:27:28.186940: step 137, loss 0.755787, acc 0.578125
2018-11-10T20:27:29.265766: step 138, loss 0.769049, acc 0.59375
2018-11-10T20:27:30.342994

2018-11-10T20:29:45.498204: step 248, loss 0.575989, acc 0.703125
2018-11-10T20:29:46.656095: step 249, loss 0.610805, acc 0.71875
2018-11-10T20:29:47.888840: step 250, loss 0.496346, acc 0.796875
2018-11-10T20:29:49.094075: step 251, loss 0.668201, acc 0.734375
2018-11-10T20:29:50.257802: step 252, loss 0.467178, acc 0.828125
2018-11-10T20:29:51.407964: step 253, loss 0.559479, acc 0.703125
2018-11-10T20:29:52.571377: step 254, loss 0.61079, acc 0.640625
2018-11-10T20:29:53.793414: step 255, loss 0.569403, acc 0.71875
2018-11-10T20:29:54.946408: step 256, loss 0.551342, acc 0.75
2018-11-10T20:29:56.211558: step 257, loss 0.5335, acc 0.78125
2018-11-10T20:29:57.665062: step 258, loss 0.475019, acc 0.765625
2018-11-10T20:29:59.040233: step 259, loss 0.538857, acc 0.765625
2018-11-10T20:30:00.562638: step 260, loss 0.501537, acc 0.765625
2018-11-10T20:30:01.940924: step 261, loss 0.445458, acc 0.765625
2018-11-10T20:30:03.484776: step 262, loss 0.497148, acc 0.765625
2018-11-10T20:30:04.

2018-11-10T20:32:25.187569: step 372, loss 0.329288, acc 0.796875
2018-11-10T20:32:26.400010: step 373, loss 0.261847, acc 0.859375
2018-11-10T20:32:27.584201: step 374, loss 0.156561, acc 0.953125
2018-11-10T20:32:28.741959: step 375, loss 0.167503, acc 0.921875
2018-11-10T20:32:29.841848: step 376, loss 0.27352, acc 0.84375
2018-11-10T20:32:30.953013: step 377, loss 0.317662, acc 0.84375
2018-11-10T20:32:32.058054: step 378, loss 0.227187, acc 0.921875
2018-11-10T20:32:33.152027: step 379, loss 0.196595, acc 0.921875
2018-11-10T20:32:34.301177: step 380, loss 0.30848, acc 0.875
2018-11-10T20:32:35.764825: step 381, loss 0.249158, acc 0.921875
2018-11-10T20:32:36.963613: step 382, loss 0.159349, acc 0.953125
2018-11-10T20:32:38.123998: step 383, loss 0.101787, acc 0.984375
2018-11-10T20:32:39.269474: step 384, loss 0.120039, acc 0.96875
2018-11-10T20:32:40.406071: step 385, loss 0.182654, acc 0.9375
2018-11-10T20:32:41.643706: step 386, loss 0.22383, acc 0.890625
2018-11-10T20:32:42.8

2018-11-10T20:35:00.161075: step 496, loss 0.103045, acc 0.96875
2018-11-10T20:35:01.274079: step 497, loss 0.0859102, acc 0.984375
2018-11-10T20:35:02.373324: step 498, loss 0.0690213, acc 0.96875
2018-11-10T20:35:03.471839: step 499, loss 0.0605011, acc 0.984375
2018-11-10T20:35:04.571220: step 500, loss 0.0615845, acc 0.984375

Evaluation:
2018-11-10T20:35:05.384629: step 500, loss 0.659426, acc 0.775

Saved model checkpoint to /Users/cosgalaxy/Desktop/cnn/runs/1541903085/checkpoints/model-500

2018-11-10T20:35:06.988548: step 501, loss 0.104937, acc 0.96875
2018-11-10T20:35:08.110660: step 502, loss 0.0342855, acc 1
2018-11-10T20:35:09.241911: step 503, loss 0.04542, acc 0.984375
2018-11-10T20:35:10.341939: step 504, loss 0.083898, acc 0.984375
2018-11-10T20:35:11.486289: step 505, loss 0.0782172, acc 0.96875
2018-11-10T20:35:12.589201: step 506, loss 0.080156, acc 0.96875
2018-11-10T20:35:13.682706: step 507, loss 0.0569857, acc 1
2018-11-10T20:35:14.777441: step 508, loss 0.05898

...

2018-11-10T20:53:09.327711: step 1400, loss 0.00163561, acc 1

Evaluation:
2018-11-10T20:53:10.190744: step 1400, loss 0.807808, acc 0.8

Saved model checkpoint to /Users/cosgalaxy/Desktop/cnn/runs/1541903085/checkpoints/model-1400

2018-11-10T20:53:12.149628: step 1401, loss 0.0025988, acc 1
2018-11-10T20:53:13.246073: step 1402, loss 0.000509179, acc 1
2018-11-10T20:53:14.342274: step 1403, loss 0.00102315, acc 1
2018-11-10T20:53:15.443713: step 1404, loss 0.00198676, acc 1
2018-11-10T20:53:16.554091: step 1405, loss 0.000326051, acc 1
2018-11-10T20:53:17.677075: step 1406, loss 0.00300801, acc 1
2018-11-10T20:53:18.828462: step 1407, loss 0.000464691, acc 1
2018-11-10T20:53:19.982524: step 1408, loss 0.00723294, acc 1
2018-11-10T20:53:21.147719: step 1409, loss 0.000483937, acc 1
2018-11-10T20:53:22.302791: step 1410, loss 0.00342813, acc 1
2018-11-10T20:53:23.476125: step 1411, loss 0.00223554, acc 1
2018-11-10T20:53:24.630430: step 1412, loss 0.00112582, acc 1
2018-11-10T20:53:25.

2018-11-10T20:55:39.272338: step 1526, loss 0.000695381, acc 1
2018-11-10T20:55:40.419029: step 1527, loss 0.000470463, acc 1
2018-11-10T20:55:41.595776: step 1528, loss 0.00101624, acc 1
2018-11-10T20:55:42.759969: step 1529, loss 0.00187971, acc 1
2018-11-10T20:55:43.913174: step 1530, loss 0.000890815, acc 1
2018-11-10T20:55:45.065206: step 1531, loss 0.00281008, acc 1
2018-11-10T20:55:46.377370: step 1532, loss 0.000688242, acc 1
2018-11-10T20:55:47.546609: step 1533, loss 0.000703454, acc 1
2018-11-10T20:55:48.701105: step 1534, loss 0.000689714, acc 1
2018-11-10T20:55:49.799915: step 1535, loss 0.00130288, acc 1
2018-11-10T20:55:50.894610: step 1536, loss 0.000687788, acc 1
2018-11-10T20:55:52.040951: step 1537, loss 0.000682607, acc 1
2018-11-10T20:55:53.206751: step 1538, loss 0.00143531, acc 1
2018-11-10T20:55:54.367922: step 1539, loss 0.00121839, acc 1
2018-11-10T20:55:55.536029: step 1540, loss 0.000486725, acc 1
2018-11-10T20:55:56.713867: step 1541, loss 0.000328693, acc 

2018-11-10T20:58:10.276701: step 1655, loss 0.000451652, acc 1
2018-11-10T20:58:11.426394: step 1656, loss 0.000413971, acc 1
2018-11-10T20:58:12.564920: step 1657, loss 0.000160378, acc 1
2018-11-10T20:58:13.694892: step 1658, loss 0.000436291, acc 1
2018-11-10T20:58:14.837540: step 1659, loss 0.00285808, acc 1
2018-11-10T20:58:15.969425: step 1660, loss 0.000821122, acc 1
2018-11-10T20:58:17.105272: step 1661, loss 0.000862827, acc 1
2018-11-10T20:58:18.233554: step 1662, loss 0.000597699, acc 1
2018-11-10T20:58:19.366545: step 1663, loss 0.00209692, acc 1
2018-11-10T20:58:20.507519: step 1664, loss 0.000212658, acc 1
2018-11-10T20:58:21.582804: step 1665, loss 0.00703722, acc 1
2018-11-10T20:58:22.655531: step 1666, loss 0.000461878, acc 1
2018-11-10T20:58:23.765269: step 1667, loss 0.00590137, acc 1
2018-11-10T20:58:24.900740: step 1668, loss 0.00689164, acc 1
2018-11-10T20:58:26.033203: step 1669, loss 0.00245714, acc 1
2018-11-10T20:58:27.166378: step 1670, loss 0.00184916, acc 1

2018-11-10T21:00:39.437312: step 1784, loss 0.000358915, acc 1
2018-11-10T21:00:40.610262: step 1785, loss 0.00065121, acc 1
2018-11-10T21:00:41.803548: step 1786, loss 0.00113449, acc 1
2018-11-10T21:00:42.967650: step 1787, loss 0.00108166, acc 1
2018-11-10T21:00:44.124340: step 1788, loss 0.000372969, acc 1
2018-11-10T21:00:45.206804: step 1789, loss 0.00106938, acc 1
2018-11-10T21:00:46.348901: step 1790, loss 0.000285907, acc 1
2018-11-10T21:00:47.492902: step 1791, loss 0.00134579, acc 1
2018-11-10T21:00:48.624961: step 1792, loss 0.000575532, acc 1
2018-11-10T21:00:49.759777: step 1793, loss 0.00182738, acc 1
2018-11-10T21:00:50.893778: step 1794, loss 0.00055043, acc 1
2018-11-10T21:00:52.024509: step 1795, loss 0.0027352, acc 1
2018-11-10T21:00:53.187504: step 1796, loss 0.000128887, acc 1
2018-11-10T21:00:54.368295: step 1797, loss 0.000788345, acc 1
2018-11-10T21:00:55.508578: step 1798, loss 0.000462502, acc 1
2018-11-10T21:00:56.641437: step 1799, loss 0.000254963, acc 1
2

2018-11-10T21:03:05.381955: step 1910, loss 0.000590932, acc 1
2018-11-10T21:03:06.482120: step 1911, loss 0.000379937, acc 1
2018-11-10T21:03:07.559406: step 1912, loss 0.000478604, acc 1
2018-11-10T21:03:08.688854: step 1913, loss 0.000629691, acc 1
2018-11-10T21:03:09.816971: step 1914, loss 0.00024482, acc 1
2018-11-10T21:03:10.971906: step 1915, loss 0.000573635, acc 1
2018-11-10T21:03:12.110732: step 1916, loss 0.00071775, acc 1
2018-11-10T21:03:13.248178: step 1917, loss 0.000306766, acc 1
2018-11-10T21:03:14.381244: step 1918, loss 0.000200502, acc 1
2018-11-10T21:03:15.514133: step 1919, loss 0.000139146, acc 1
2018-11-10T21:03:16.623452: step 1920, loss 0.000114388, acc 1
2018-11-10T21:03:17.716576: step 1921, loss 0.000188444, acc 1
2018-11-10T21:03:18.840571: step 1922, loss 0.00138591, acc 1
2018-11-10T21:03:19.971988: step 1923, loss 0.000228018, acc 1
2018-11-10T21:03:21.107924: step 1924, loss 0.000879726, acc 1
2018-11-10T21:03:22.239822: step 1925, loss 0.000325738, a